### Introduction

This project is aimed to practice using SQL to answer business questions.

For it I'm going to use the database of the imaginary music store (similar to iTunes) which is called The Chinook.

Here the first steps for preparing to use the database:

In [2]:
import sqlite3
import pandas as pd

Create a function, that takes an DQL query and returns a pandas dataframe:

In [3]:
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)

Create a function, that takes an DML, DDL, etc. query and executes it:

In [4]:
def run_command(query):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(query)

Create a function to return a list off all tables and views:

In [12]:
def show_tables():
    query = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    print(run_query(query))

Let's look at the tables and views:

In [15]:
show_tables()

              name   type
0            album  table
1           artist  table
2         customer  table
3         employee  table
4            genre  table
5          invoice  table
6     invoice_line  table
7       media_type  table
8         playlist  table
9   playlist_track  table
10           track  table


### Selecting albums to purchase

The Chinook record store has just signed a deal with a new record label. The label from the USA and it has three albums of the different genres. 

It was decided to add only three of them to the store.

So it is needed to find out which genres sells the most tracks in the USA.

In [9]:
genre_sales_usa_query = '''
WITH sold_tracks_usa AS (
    SELECT 
        c.country,
        SUM(il.quantity) AS sold_tracks_usa_total
    FROM invoice_line AS il
        INNER JOIN invoice AS i 
        ON il.invoice_id = i.invoice_id
        INNER JOIN customer AS c 
        ON i.customer_id = c.customer_id
    WHERE c.country = 'USA'
    GROUP BY 1
)

SELECT 
g.name AS genre,
SUM(il.quantity) AS sold_tracks_tally,
CAST(SUM(il.quantity) AS FLOAT) / u.sold_tracks_usa_total AS sold_tracks_percent
FROM genre AS g
    INNER JOIN track AS t 
    ON g.genre_id = t.genre_id
    INNER JOIN invoice_line AS il 
    ON t.track_id = il.track_id
    INNER JOIN invoice AS i 
    ON il.invoice_id = i.invoice_id
    INNER JOIN customer AS c 
    ON i.customer_id = c.customer_id
    INNER JOIN sold_tracks_usa AS u
    ON c.country = u.country
WHERE c.country = 'USA'
GROUP BY 1
ORDER BY 2 DESC;
'''

run_query(genre_sales_usa_query)

,genre,sold_tracks_tally,sold_tracks_percent
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


Four albums of the new record label have different genres:
 - Hip-Hop
 - Punk
 - Pop
 - Blues
 
From the table above it is easily seen that the least popular genre from the four is "Hip-Hop". Although it is not so far behind.
 
Now I'd recommend to focus on the more popular "Punk", "Blues"and "Pop".

### Analyzing Employee Sales Perfomance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.

Firstly, it is needed to determine their sales perfomance, and secondly alanyze any other useful attributes.

In [18]:
employee_perfomance_query = '''
SELECT
    e.first_name || ' ' || e.last_name AS employee_name,
    SUM(i.total) AS sales_dollar_total,
    COUNT(DISTINCT c.customer_id) AS customer_tally,
    SUM(i.total) / COUNT(DISTINCT c.customer_id) AS sales_per_customer,
    COUNT(DISTINCT i.invoice_id) AS invoice_tally,
    SUM(i.total) / COUNT(DISTINCT i.invoice_id) AS sales_per_invoice
FROM employee AS e
    INNER JOIN customer AS c
    ON e.employee_id = c.support_rep_id
    
    INNER JOIN invoice AS i
    ON c.customer_id = i.customer_id   
GROUP BY 1
ORDER BY 2 DESC;
'''

run_query(employee_perfomance_query)

,employee_name,sales_dollar_total,customer_tally,sales_per_customer,invoice_tally,sales_per_invoice
0,Jane Peacock,1731.51,21,82.452857,212,8.167500
1,Margaret Park,1584.00,20,79.200000,214,7.401869
2,Steve Johnson,1393.92,18,77.440000,188,7.414468


So I've counted total sales for each employee. Also I decided to count the number of customers for each employee and the number of invoices.

Then I counted average sale for one customer and for one invoice.

It is seen that although Steve Johnson has less total sales than Margaret Park, the average invoice of his average client is bigger than the same of Margaret Park.

Jane Peacock is a top performer in the every indicator.

### Analyzing Sales by Country



It is needed to analyze the sales data for customers from each different country.

In [19]:
countries_sales = '''
WITH 
country_aggregates AS (
    SELECT
        c.country,
        COUNT(DISTINCT c.customer_id) AS customers_total,
        SUM(i.total) AS sales_total,
        SUM(i.total) / COUNT(DISTINCT c.customer_id) AS sales_per_customer,
        AVG(i.total) AS average_invoice_value
    FROM customer AS c
        INNER JOIN invoice AS i
        ON c.customer_id = i.customer_id
    GROUP BY 1),

/* Change the name of the country with only one customer
to 'Other' and group these countries*/

country_aggregates_with_others AS (
    SELECT
        CASE
            WHEN customers_total > 1 THEN country
            ELSE 'Other'
        END AS country,
        SUM(customers_total) AS customers_total,
        SUM(sales_total) AS sales_total,
        SUM(sales_per_customer) AS sales_per_customer,
        SUM(average_invoice_value) AS average_invoice_value,
        CASE
            WHEN customers_total = 1 THEN 1
            ELSE 0
        END AS other -- Add the flag for the further sorting
    FROM country_aggregates
    GROUP BY 1)

SELECT
    country,
    customers_total,
    sales_total,
    sales_per_customer,
    average_invoice_value
FROM country_aggregates_with_others
ORDER BY other ASC, sales_total DESC
;
'''

run_query(countries_sales)

,country,customers_total,sales_total,sales_per_customer,average_invoice_value
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,1094.940000,111.676066


It is seen from the data that North America is a prime market and it is following up by Brazil, Canada and France, which have a good potential for growth.

### Albums vs Individual Tracks Analysis



In [32]:
albums_vs_tracks_query = '''
WITH invoice2track AS (
    SELECT i.invoice_id, t.track_id, t.album_id
    FROM invoice AS i
             INNER JOIN invoice_line AS il
                        ON i.invoice_id = il.invoice_id

             INNER JOIN track AS t
                        ON il.track_id = t.track_id
),

     album2track AS (
         SELECT a.album_id, t.track_id
         FROM album AS a
                  INNER JOIN track AS t
                             ON a.album_id = t.album_id
     ),

     invoice_album_purchase_status AS (
         SELECT i.invoice_id,
                CASE
                    WHEN (
                            (
                                SELECT i2t.track_id
                                FROM invoice2track AS i2t
                                WHERE i.invoice_id = i2t.invoice_id
                                    
                                EXCEPT

                                SELECT a2t.track_id
                                FROM album2track AS a2t
                                WHERE a2t.album_id = (
                                    SELECT MAX(i2t2.album_id)
                                    FROM invoice2track AS i2t2
                                    WHERE i2t2.invoice_id = i.invoice_id
                                    GROUP BY i2t2.invoice_id
                                )
                            ) IS NULL
                            AND
                            (
                                SELECT a2t.track_id
                                FROM album2track AS a2t
                                WHERE a2t.album_id = (
                                    SELECT MAX(i2t2.album_id)
                                    FROM invoice2track AS i2t2
                                    WHERE i2t2.invoice_id = i.invoice_id
                                    GROUP BY i2t2.invoice_id
                                )
                                    EXCEPT

                                SELECT i2t.track_id
                                FROM invoice2track AS i2t
                                WHERE i.invoice_id = i2t.invoice_id
                            ) IS NULL
                        ) THEN 1
                    ELSE 0
                    END AS album_purchase_status

         FROM invoice AS i
         ORDER BY 1)

SELECT album_purchase_status,
       COUNT(*)                                             AS total_number,
       ROUND(CAST(COUNT(*) AS FLOAT) * 100 /
       (SELECT COUNT(*) FROM invoice_album_purchase_status), 2) AS share_percent
FROM invoice_album_purchase_status
GROUP BY 1
;

'''

run_query(albums_vs_tracks_query)

,album_purchase_status,total_number,share_percent
0,0,500,81.43
1,1,114,18.57


Full album purchases represent less than one fifth of all purchases. So purchasing only the most popular tracks from record companies could be considered as a good strategy.